In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os
from scipy import ndimage, spatial
from skimage.color import rgb2gray
from skimage.io import imread
from scipy.stats import multivariate_normal
from scipy.ndimage.filters import convolve
import numpy.linalg as LA
import copy
import time
import threading
from multiprocessing import Process 

In [2]:
def read_directory(directory_name):
    filenumber = len([name for name in os.listdir(directory_name) if os.path.isfile(os.path.join(directory_name, name))])
    for i in range(1,filenumber+1):
        img = cv2.imread(directory_name + "/" + str(i)+".jpg")
        array_of_img.append(img)

In [3]:
def gaussian(sigma): 
    size = 2*np.ceil(3*sigma)+1
    x, y = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1] 
    g = np.exp(-((x**2 + y**2)/(2.0*sigma**2))) / (2*np.pi*sigma**2)
    return g/g.sum()

In [4]:
def get_gaussian(img,sig):
    image = np.zeros((img.shape[0],img.shape[1],len(sig)))
    for i in range(len(sig)):
        image[:,:,i] = cv2.GaussianBlur(img, None, sig[i])
    return image

In [5]:
def get_dog(img):
    dog = np.zeros((img.shape[0],img.shape[1],img.shape[2]-1))
    for i in range(img.shape[2]-1):
        dog[:,:,i] = img[:,:,i+1] - img[:,:,i]
    return dog

In [6]:
def localize_keypoint(D, x, y, s):
    dx = (D[y,x+1,s]-D[y,x-1,s])/2.
    dy = (D[y+1,x,s]-D[y-1,x,s])/2.
    ds = (D[y,x,s+1]-D[y,x,s-1])/2.

    dxx = D[y,x+1,s]-2*D[y,x,s]+D[y,x-1,s]
    dxy = ((D[y+1,x+1,s]-D[y+1,x-1,s]) - (D[y-1,x+1,s]-D[y-1,x-1,s]))/4.
    dxs = ((D[y,x+1,s+1]-D[y,x-1,s+1]) - (D[y,x+1,s-1]-D[y,x-1,s-1]))/4.
    dyy = D[y+1,x,s]-2*D[y,x,s]+D[y-1,x,s]
    dys = ((D[y+1,x,s+1]-D[y-1,x,s+1]) - (D[y+1,x,s-1]-D[y-1,x,s-1]))/4.
    dss = D[y,x,s+1]-2*D[y,x,s]+D[y,x,s-1]

    J = np.array([dx, dy, ds])
    HD = np.array([
        [dxx, dxy, dxs],
        [dxy, dyy, dys],
        [dxs, dys, dss]]) 
    
    offset = LA.lstsq(HD, J)[0]
        
    return offset, J, HD[:2,:2]

In [7]:
def get_keypoints(D,trim):
    extreme = []
    keypoints = []
    low_contrast = 0
    too_much_offset = 0
    edge_response = 0
    for s in range(1, D.shape[2]-1):
        for y in range(trim, D.shape[0] - trim):
            for x in range(trim, D.shape[1] - trim):
                patch = D[y-1:y+2, x-1:x+2, s-1:s+2]
                
                if np.argmax(patch) == 13 or np.argmin(patch) == 13: 
                    
                    offset, J, H = localize_keypoint(D, x, y, s)
                    contrast = D[y,x,s] + .5*J.dot(offset)
                    
                    if abs(contrast) < 0.03: 
                        low_contrast += 1
                        continue
                    if max(np.absolute(offset)>0.5): 
                        too_much_offset += 1
                        continue

                    w, v = LA.eig(H)
                    r = w[1]/w[0]
                    R = (r+1)**2 / r
                    if R > (10+1)**2 / 10.0: 
                        edge_response += 1
                        continue

                    kp = np.array([x, y, s]) + offset
                    if kp[1] >= D.shape[0] or kp[0] >= D.shape[1]: continue 

                    keypoints.append(kp)
    return np.array(keypoints)

In [8]:
def get_orientation(kps, octave):
    new_kps = []
    bin_width = 360//36

    for kp in kps:
        cx, cy, s = int(kp[0]), int(kp[1]), int(kp[2])
        s = np.clip(s, 0, octave.shape[2]-1)

        sigma = kp[2]*1.5
        w = int(2*np.ceil(sigma)+1)
        kernel = gaussian(sigma)
        
        L = octave[...,s]
        hist = np.zeros(36, dtype=np.float32)

        for oy in range(-w, w+1):
            for ox in range(-w, w+1):
                x, y = cx+ox, cy+oy
                
                if x < 0 or x > octave.shape[1]-1: 
                    continue
                elif y < 0 or y > octave.shape[0]-1: 
                    continue           
                
                dy = L[min(L.shape[0]-1, y+1),x] - L[max(0, y-1),x]
                dx = L[y,min(L.shape[1]-1, x+1)] - L[y,max(0, x-1)]
                magnitude = np.sqrt(dx**2+dy**2)
                theta = (np.arctan2(dy,dx)+np.pi) * 180/np.pi
                weight = kernel[min(oy+w,kernel.shape[0]-1), min(ox+w,kernel.shape[1]-1)] * magnitude
                bin_idx = np.floor(theta)//bin_width
                hist[int(bin_idx)%36] += weight

        max_bin = np.argmax(hist)
        new_kps.append([kp[0], kp[1], kp[2], fit(hist, max_bin, bin_width)])

        max_val = np.max(hist)
        for binno, val in enumerate(hist):
            if binno == max_bin: 
                continue
            if .8 * max_val <= val:
                new_kps.append([kp[0], kp[1], kp[2], fit(hist, binno, bin_width)])

    return np.array(new_kps)

In [9]:
def fit(hist, binnum, bin_width):
    centerval = binnum*bin_width + bin_width/2.

    if binnum == len(hist)-1: 
        rightval = 360 + bin_width/2.
    else: 
        rightval = (binnum+1)*bin_width + bin_width/2.

    if binnum == 0: 
        leftval = -bin_width/2.
    else: 
        leftval = (binnum-1)*bin_width + bin_width/2.
    
    A = np.array([
        [centerval**2, centerval, 1],
        [rightval**2, rightval, 1],
        [leftval**2, leftval, 1]])
    b = np.array([
        hist[binnum],
        hist[(binnum+1)%len(hist)], 
        hist[(binnum-1)%len(hist)]])

    x = LA.lstsq(A, b, rcond=None)[0]
    if x[0] == 0: 
        x[0] = 1e-6
    return -x[1]/(2*x[0])

In [10]:
def get_histogram(m, theta, reference_angle):
    bin_width = 360 // 8
    hist = np.zeros(8, dtype=np.float32)
    c = 4/2 - .5

    for i, (mag, angle) in enumerate(zip(m, theta)):
        angle = (angle-reference_angle) % 360
        bin_idx = np.floor(angle)//bin_width
        
        vote = mag
        hist_interp_weight = 1 - abs(angle - (bin_idx*bin_width + bin_width/2))/(bin_width/2)
        vote *= max(hist_interp_weight, 1e-6)

        gy, gx = np.unravel_index(i, (4, 4))
        x_interp_weight = max(1 - abs(gx - c)/c, 1e-6)
        y_interp_weight = max(1 - abs(gy - c)/c, 1e-6)
        vote *= x_interp_weight * y_interp_weight

        hist[int(bin_idx)%8] += vote

    return hist

In [11]:
def get_descriptors(kps, octave):
    descs = []
    bin_width = 360//8

    for kp in kps:
        cx, cy, s = int(kp[0]), int(kp[1]), int(kp[2])
        s = np.clip(s, 0, octave.shape[2]-1)
        L = octave[...,s]
        t, l = max(0, cy-8), max(0, cx-8)
        b, r = min(L.shape[0], cy+9), min(L.shape[1], cx+9)
        patch = L[t:b, l:r]
        
        r1 = np.zeros_like(patch)
        r1[-1] = patch[-1]
        r1[:-1] = patch[1:]

        r2 = np.zeros_like(patch)
        r2[0] = patch[0]
        r2[1:] = patch[:-1]

        dy = r1-r2

        r1[:,-1] = patch[:,-1]
        r1[:,:-1] = patch[:,1:]

        r2[:,0] = patch[:,0]
        r2[:,1:] = patch[:,:-1]

        dx = r1-r2
        
        magnitude = np.sqrt(dx**2+dy**2)
        theta = (np.arctan2(dy,dx)+np.pi) * 180/np.pi

        featvec = np.zeros(8 * 4**2, dtype=np.float32)

        for i in range(0, 4):
            for j in range(0, 4):
                t, l = i*4, j*4
                b, r = min(L.shape[0], (i+1)*4), min(L.shape[1], (j+1)*4)

                hist = get_histogram(magnitude[t:b, l:r].ravel(), theta[t:b, l:r].ravel(), kp[3])
                
                featvec[i*4*8 + j*8 : i*4*8 + (j+1)*8] = hist.flatten()

        featvec /= max(1e-6, LA.norm(featvec))
        featvec[featvec>0.2] = 0.2
        featvec /= max(1e-6, LA.norm(featvec))
        descs.append(featvec)

    return np.array(descs)

In [12]:
def get_matches(kpsA, kpsB, feats1, feats2, ratio=0.8):
    idxs1, idxs2 = [], []
    for i, feat in enumerate(feats1):
        distances = LA.norm(feats2-feat, axis=1)
        nn = np.argsort(distances)[:2]
        dist1, dist2 = distances[nn[0]], distances[nn[1]]

        if dist1/max(1e-6, dist2) < ratio:
            idxs1.append(i)
            idxs2.append(nn[0])
            
    ptsA = kpsA[idxs1,:-2]
    ptsB = kpsB[idxs2,:-2]
    (H, status) = cv2.estimateAffinePartial2D(ptsA, ptsB)
    return H

In [13]:
s = 3
k = 2**(1./s)
sigma = []
for i in range(-1,14):
    sigma.append(1.6*(k**i))
sigma[0] = 1.3
SAVING_EVENT = threading.Event()
SAVING_EVENT.set()

def process_octave(img,i,kps,descriptors):
    scale = 2**i
    gaussian = get_gaussian(img,sigma[i*3:i*3+6])
    if scale != 1:
        gaussian =  cv2.resize(gaussian,None,fx=1/scale,fy=1/scale)
    DoG = get_dog(gaussian)
    kp = get_keypoints(DoG,16//scale)
    kp = get_orientation(kp,DoG)
    des = get_descriptors(kp,DoG)
    kp *= scale
    SAVING_EVENT.wait()
    SAVING_EVENT.clear()
    if len(kp) != 0:
        descriptors.append(des)
        kps.append(kp)
    SAVING_EVENT.set()

In [14]:
def cylindricalWarp(img,f):
    h_,w_ = img.shape[:2]
    K = np.array([[f,0,w_/2],[0,f,h_/2],[0,0,1]]) # mock intrinsics
    y_i, x_i = np.indices((h_,w_))
    X = np.stack([x_i,y_i,np.ones_like(x_i)],axis=-1).reshape(h_*w_,3) # to homog
    Kinv = np.linalg.inv(K) 
    X = Kinv.dot(X.T).T
    A = np.stack([np.sin(X[:,0]),X[:,1],np.cos(X[:,0])],axis=-1).reshape(w_*h_,3)
    B = K.dot(A.T).T
    B = B[:,:-1] / B[:,[-1]]
    B[(B[:,0] < 0) | (B[:,0] >= w_) | (B[:,1] < 0) | (B[:,1] >= h_)] = -1
    B = B.reshape(h_,w_,-1)
    result = cv2.remap(img, B[:,:,0].astype(np.float32), B[:,:,1].astype(np.float32), cv2.INTER_AREA, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    trim = 8
    return result[trim:result.shape[0]-trim,trim:result.shape[1]-trim,:]

In [15]:
def stiching_right(kp1,kp2,im1,im2,H):
    nH = get_matches(kp1['kp'], kp2['kp'],kp1['des'], kp2['des'])
    if H is None:
        H = np.append(nH,np.array([[0,0,1]]),axis=0)
    else:
        H = np.dot(H,np.append(nH,np.array([[0,0,1]]),axis=0))
    new_size = (im1.shape[1] + im2.shape[1], im1.shape[0])
    background = cv2.copyMakeBorder(im2,0,0,0,im1.shape[1],cv2.BORDER_CONSTANT,value=[0,0,0])
    
    result = cv2.warpAffine(im1, H[:2], new_size,background, borderMode=cv2.BORDER_TRANSPARENT)
        
    return result,H

In [16]:
def stiching_left(kp1,kp2,im1,im2,H):
    nH = get_matches(kp1['kp'], kp2['kp'],kp1['des'], kp2['des'])
    if H is None:
        H = np.append(nH,np.array([[0,0,1]]),axis=0)
    else:
        H = np.dot(H,np.append(nH,np.array([[0,0,1]]),axis=0))
        
    H[0][2] += im1.shape[1]
    new_size = (im1.shape[1] + im2.shape[1], im1.shape[0])
    background = cv2.copyMakeBorder(im2,0,0,im1.shape[1],0,cv2.BORDER_CONSTANT,value=[0,0,0])
    
    result = cv2.warpAffine(im1, H[:2], new_size,background, borderMode=cv2.BORDER_TRANSPARENT)
        
    return result,H

In [17]:
def trim(img):
    y_min, y_max, x_min, x_max = 0,0,0,0
    for i in range(img.shape[0]):
        if np.sum(img[i,:,0]) > 0:
            y_min = i
            break

    for i in range(img.shape[0]-1,0,-1):
        if np.sum(img[i,:,0]) > 0:
            y_max = i
            break

    for i in range(img.shape[1]):
        if np.sum(img[:,i,0]) > 0:
            x_min = i
            break        

    for i in range(img.shape[1]-1,0,-1):
        if np.sum(img[:,i,0]) > 0:
            x_max = i
            break
    img = img[y_min:y_max,x_min:x_max,:]
    return img

In [18]:
def SIFT(inputname):
    print(inputname)
    read_directory(inputname)
    #use: len(array_of_img) for looping the image, array_of_img[0],
    #array_of_img[1],array_of_img[2],...for processing each image
    #Start SIFT here
    im_sets = []
    for im in array_of_img:
        im = cylindricalWarp(im,765)
        img = cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
        kps = []
        descriptors = []
        octave = 4
        
        threads = []
        for i in range(octave):
            threads.append(threading.Thread(target=process_octave, args=[img,i,kps,descriptors]))
            threads[i].start()
        
        for i in range(octave):
            threads[i].join()        
        
        kps = np.vstack(kps)
        descriptors = np.vstack(descriptors)
        
        im = cv2.copyMakeBorder(im,200,200,0,0,cv2.BORDER_CONSTANT,value=[0,0,0])
        kp_set = {'kp':kps,'des':descriptors}
        im_sets.append({'image':im, 'kp_set':kp_set})
    
    img_num =len(im_sets)
    imageout = im_sets[img_num//2]['image']
    
    H = None
    for i in range(img_num//2,img_num-1) :
        imageout, H = stiching_right(im_sets[i+1]['kp_set'],im_sets[i]['kp_set'],im_sets[i+1]['image'],imageout,H) 
        
    H = None
    last_set = im_sets[img_num//2]
    for i,imset in enumerate(reversed(im_sets[:img_num//2])):
        imageout, H = stiching_left(imset['kp_set'],last_set['kp_set'],imset['image'],imageout,H) 
        last_set = imset
    
    imageout = trim(imageout)
        
    #End of SIFT here and use imageoutput for your output
    cv2.imwrite(inputname+'.jpg', imageout)
    array_of_img.clear()
    print(inputname,'finish')

In [20]:
f = open('testfile.txt', 'r')
dirname = str(f.readline()).strip()
start_time = time.time()
array_of_img = [] # Store all the image data
process = []

while(dirname):
    p = Process(target=SIFT,args=(dirname,))
    p.start()
    process.append(p)
    dirname = str(f.readline()).strip()
    
for i in range(len(process)):
    process[i].join()

duration = int(time.time()-start_time)
print(duration//60,'m',duration%60,'s')

NCTUparrington
pano



/home/lab616/anaconda3/envs/test/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
/home/lab616/anaconda3/envs/test/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
/home/lab616/anaconda3/envs/test/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default a

pano finish


/home/lab616/anaconda3/envs/test/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in double_scalars
/home/lab616/anaconda3/envs/test/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in double_scalars
/home/lab616/anaconda3/envs/test/lib/python3.6/site-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in double_scalars


NCTU finish
parrington finish
54 m 49 s
